### 트랜스포머의 잠재적 문제점

규모가 매우 크다
- 갈력한 모델을 학습시키는데, 많은 비용이 들어간다. 이런 비용을 충당할 수 없는 기관은 연구 범위가 제한될 수 밖에 없다.
- 대량의 저력소모로 환경적 문제가 있다.
- 오픈소스로 모델이 제공되어도 추론 하는데에만 많은 gpu가 필요할 수 있다.

순차 처리
- 결국 첫 토큰 부터 마지막 토큰까지 순서대로 처리하는 것이기 때문에, 순차 처리로 인해 입력 길이에 대해 2차 시간 복잡도를 가지는 문제가 있다.
- 이는 처리 시간이 입력 길이의 제고벵 비례해 증가한다.

고정 입력 크기
- 모델이 처리 할 수 있는 최대 토큰 수가 정해져 있다. 이는 모델 규모에 따라 정해져 있어, 추후 늘리거나 할 수 없다.
- 모델이 처리 할 수 있는 토큰의 수는 컨텐스트 윈도우 에 따라 다르다.

이해하기 어려움
- 트랜스포머는 블랙박스다. 입력에 어떤 조작을 가해서 출력으로 내뱉는지 완벽하게 알지 못한다.

### 활용 범위 - VIT

- 최근 다양한 분야에 트랜스포머 기법이 활용되고 있다. 특히 이미지 분야에서의 활용성도 뛰어나다. 이미지 인식부터, 분할, 객체 감지, 비디오 이해 등 여러 분야에 활용되고 있다.
- 비전 트랜스포머 (vision transformer *VIT)의 도입으로 어텐션과 트랜스포머 기반 기술을 사용해 비전 문제를 해결해 나가고 있지만 CNN(convolutional neural network)를 완전히 배제하지는 않는다.
- 이미지 처리 파이프라인에서 합성곱 신경망은 높은 수준의 엣지, 텍스터 및 기타 패턴을 감지하는 feature map을 추출한다. 이는 겹치지 않는 고정 크기의 patch로 나뉘고 이는 토큰 시퀀스와 유사하다.
- 트랜스포머는 이 feature map patch간의 관계를 학습할 수 있다.



- VIT는 CNN보다 학습에 매우 많은 데이터를 필요로 한다. 예로, DeiT(data-efficient image transformer) 모델은 CNN에서 일반적으로 사용되는 augmentation(증강)과 regularization(정규화) 기법을 활용해 학습을 진행했다.
- 제로샷 이미지 분류는 트랜스포머 기반 이미지 모델의 대표적인 예다. 기존의 이미지 분류 모델은 고정된 클래스 집합으로 학습되지만, 제로샷 이미지 분류는 추론 시점에 클래스를 지정할 수 있다.

In [ ]:
# 이미지를 뭐 어디서든 불러오기
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import os

cache_dir = r"D:\WorkSpace\Hands_on_Gen_AI"
os.makedirs(cache_dir, exist_ok=True)

# model_id = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"
# tok = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_dir)
# mdl = AutoModelForSequenceClassification.from_pretrained(model_id, cache_dir=cache_dir)

# classifier = pipeline("text-classification", model=mdl, tokenizer=tok)
# print(classifier("I absolutely love this movie!"))

image = "load some image"

pipe = pipeline("zerp-shot-image-classification", model = "openai/clip-vit-base-patch32")
labels = ['cat','dog','zebra','rion']
pipe(image, cadidate_labels=labels )
image